In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd
import sys
import json
import math

ticker = "^GSPC"
start_date = "1927-12-01"
end_date = "2025-04-26"

print(f"Attempting to download historical data for {ticker} from {start_date} to {end_date}...")
sys.stdout.flush()

try:
    sp500_data = yf.download(ticker, start=start_date, end=end_date, interval="1d", actions=False)

    print("\nData downloaded successfully!")

    if isinstance(sp500_data.columns, pd.MultiIndex):
        print("\nFlattening column MultiIndex...")
        sp500_data.columns = sp500_data.columns.get_level_values(0)
        print("Columns flattened.")

    print("\nDataFrame Info after flattening columns:")
    sp500_data.info()
    print(f"\nTotal rows (trading days) downloaded: {len(sp500_data)}")
    print("\nFirst 5 rows (Earliest available data):")
    print(sp500_data.head())
    print("\nLast 5 rows (Latest available data):")
    print(sp500_data.tail())
    print("\nMissing values per column:")
    print(sp500_data.isnull().sum())

    initial_rows = len(sp500_data)
    sp500_data.ffill(inplace=True)
    sp500_data.bfill(inplace=True)
    if len(sp500_data) < initial_rows:
         print("\nNote: Rows with only NaN might have been dropped implicitly before filling.")
    elif sp500_data.isnull().sum().sum() > 0:
         print("\nWarning: Some missing values might still remain after ffill/bfill.")
    else:
         print("\nForward/Backward fill applied (if any NaNs existed).")

    csv_path_local = 'S&P_500_historical_1927_2025.csv'
    sp500_data.to_csv(csv_path_local)
    print(f"\nData saved successfully to {csv_path_local}")

    json_path_local = 'S&P_500_historical_1927_2025.json'
    sp500_data_reset = sp500_data.reset_index()
    sp500_data_reset['Date'] = sp500_data_reset['Date'].astype(str)

    print(f"\nConverting DataFrame to JSON format...")
    data_list = sp500_data_reset.to_dict(orient='records')

    for record in data_list:
        for key, value in record.items():
            if isinstance(value, float) and math.isnan(value):
                record[key] = None

    print(f"Saving data to JSON file: {json_path_local}")
    with open(json_path_local, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

    print(f"Data saved successfully to {json_path_local}")
    print("You can download both files from the Colab file browser.")

except Exception as e:
    print(f"\nAn error occurred during download or processing: {e}")
    print("Check the ticker symbol and date range, or potential network issues.")

Attempting to download historical data for ^GSPC from 1927-12-01 to 2025-04-26...


[*********************100%***********************]  1 of 1 completed


Data downloaded successfully!

Flattening column MultiIndex...
Columns flattened.

DataFrame Info after flattening columns:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24445 entries, 1927-12-30 to 2025-04-25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   24445 non-null  float64
 1   High    24445 non-null  float64
 2   Low     24445 non-null  float64
 3   Open    24445 non-null  float64
 4   Volume  24445 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 1.1 MB

Total rows (trading days) downloaded: 24445

First 5 rows (Earliest available data):
Price           Close       High        Low       Open  Volume
Date                                                          
1927-12-30  17.660000  17.660000  17.660000  17.660000       0
1928-01-03  17.760000  17.760000  17.760000  17.760000       0
1928-01-04  17.719999  17.719999  17.719999  17.719999       0
1928-01-05  17.549999  17.549999  17.549


Data saved successfully to S&P_500_historical_1927_2025.csv

Converting DataFrame to JSON format...
Saving data to JSON file: S&P_500_historical_1927_2025.json
Data saved successfully to S&P_500_historical_1927_2025.json
You can download both files from the Colab file browser.
